In [2]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *

In [3]:
run_id = 181
qid = 1281

In [4]:
docs = Doc.objects.filter(query=qid)
wcs = WC.objects.all()
topics = Topic.objects.filter(run_id_id=run_id)

In [5]:
for wc in wcs.iterator():
    print(wc.text)

Oceanography
Ecology
Chemistry, Multidisciplinary
Biodiversity Conservation
Biology
Economics
Geology
Plant Sciences
International Relations
Agriculture, Dairy & Animal Science
Agriculture, Multidisciplinary
Multidisciplinary Sciences
Energy & Fuels
Chemistry, Physical
Environmental Sciences
Parasitology
Physics, Multidisciplinary
Computer Science, Information Systems
Geosciences, Multidisciplinary
Thermodynamics
Meteorology & Atmospheric Sciences
Environmental Studies
Agronomy
Nuclear Science & Technology
Social Sciences, Interdisciplinary
Computer Science, Interdisciplinary Applications
Engineering, Electrical & Electronic
Marine & Freshwater Biology
Geography, Physical
Engineering, Environmental
Biotechnology & Applied Microbiology
Engineering, Chemical
Medicine, General & Internal
Forestry
Engineering, Mechanical
Metallurgy & Metallurgical Engineering
Engineering, Civil
Soil Science
Psychology, Developmental
Engineering, Aerospace
Agricultural Engineering
Food Science & Technology


In [6]:
tvalues = []
for t in topics:
    tv = {"t_id": t.pk, "title":t.title, "score": t.score}
    for wc in wcs.iterator():
        dts = DocTopic.objects.filter(
            run_id=run_id,topic=t,doc__wc=wc
        ).aggregate(
            total = Sum('score')
        )
       
        twcv = {
            "wc_id" : wc.id,
            "wc_text": wc.text,
            "oecd" : wc.oecd,
            "oecd_fos_text" : wc.oecd_fos_text,
            "wc_score" : dts['total']
        }
        twcv.update(tv)
        
        tvalues.append(twcv)
        

In [7]:
df = pd.DataFrame.from_dict(list(tvalues))
df.head()

,oecd,oecd_fos_text,score,t_id,title,wc_id,wc_score,wc_text
0,Natural Sciences,Earth and related environmental sciences,55.284927,20123,"{urban, land, citi}",1,0.033871,Oceanography
1,Natural Sciences,Biological sciences,55.284927,20123,"{urban, land, citi}",236,1.878638,Ecology
2,Natural Sciences,Chemical sciences,55.284927,20123,"{urban, land, citi}",237,0.069853,"Chemistry, Multidisciplinary"
3,Natural Sciences,Biological sciences,55.284927,20123,"{urban, land, citi}",238,0.744505,Biodiversity Conservation
4,Natural Sciences,Biological sciences,55.284927,20123,"{urban, land, citi}",239,0.095993,Biology


In [8]:
pdf = df.pivot(index='title', columns='wc_text', values='wc_score')

In [9]:
pdf

wc_text,Acoustics,Agricultural Economics & Policy,Agricultural Engineering,"Agriculture, Dairy & Animal Science","Agriculture, Multidisciplinary",Agronomy,Allergy,Anatomy & Morphology,Andrology,Anesthesiology,...,Transportation,Transportation Science & Technology,Tropical Medicine,Urban Studies,Urology & Nephrology,Veterinary Sciences,Virology,Water Resources,Women's Studies,Zoology
title,,,,,,,,,,,,,,,,,,,,,
"{activ, school, health}",0.018529,0.060487,0.042189,NaN,0.057053,0.008435,0.100241,NaN,NaN,0.004202,...,0.889142,0.205187,NaN,0.266081,NaN,0.059557,NaN,0.092236,0.027255,0.035739
"{air, pollut, qualiti}",0.115082,NaN,0.088074,NaN,0.047564,0.021256,0.102643,NaN,NaN,NaN,...,0.413291,0.168046,NaN,0.158726,NaN,0.068981,NaN,0.068990,NaN,0.003367
"{bicycl, cycl, cyclist}",NaN,NaN,NaN,NaN,NaN,0.002368,NaN,NaN,NaN,NaN,...,0.814341,0.100338,NaN,0.268052,NaN,NaN,NaN,0.003336,NaN,NaN
"{brt, bus, system}",NaN,NaN,0.002083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.176525,0.738970,NaN,0.013728,NaN,NaN,NaN,0.001323,NaN,NaN
"{bus, servic, oper}",0.001198,NaN,0.005092,NaN,NaN,0.040037,NaN,NaN,NaN,0.013510,...,1.950021,1.367686,NaN,0.417832,NaN,0.002500,NaN,0.034665,NaN,NaN
"{climat, chang, mitig}",NaN,0.039672,0.101071,NaN,0.023586,0.027618,0.062827,NaN,NaN,NaN,...,0.370670,0.038841,NaN,0.803351,0.010258,0.001101,NaN,0.105683,0.010593,0.002719
"{develop, sustain, plan}",0.082083,0.088418,0.220569,NaN,0.132830,0.064630,0.022273,NaN,NaN,0.005485,...,3.444645,1.543915,NaN,2.430828,0.045461,0.042035,NaN,0.487486,0.012311,0.018316
"{emiss, carbon, ghg}",NaN,0.088291,0.177897,NaN,0.094551,0.049329,0.010250,NaN,NaN,NaN,...,0.328376,0.069769,NaN,0.478514,0.004561,NaN,NaN,0.069285,NaN,0.004795
"{energi, build, consumpt}",0.041664,0.004610,0.069815,NaN,0.007429,0.015310,0.018949,NaN,NaN,NaN,...,0.174132,0.125250,NaN,0.115740,0.010791,0.018432,NaN,0.073019,NaN,0.002759


In [10]:
import csv

with open('../data/topic_disciplines.csv', 'w', newline='') as csvfile:
    w = csv.writer(csvfile,delimiter='\t')
    

    for name, group in df.groupby('title'):
        w.writerow(['#########################'])
        w.writerow([name])
        g = group.sort_values(by='wc_score', ascending=False).reset_index(drop=True)
        #for gr in g:
        #    w.writerow()
        for i in range(5):
            w.writerow([g['score'][i],g['wc_text'][i],g['wc_score'][i]])
               
        w.writerow([])
    
    

In [11]:
g['score'][0]


22.3725359284972